
# ProteinDT (Hugging Face) — CoT Generation

- Sample dataset
- Generate CoT (Using GPT) with groud truth (Large Scale)


In [ ]:
#@title Mount Google Drive
USE_DRIVE = True
DRIVE_SUBDIR = "hf/proteinDT"
BASE_DIR = ""


if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    from pathlib import Path as _Path
    _drive_base = _Path('')
    BASE_DIR = _drive_base / DRIVE_SUBDIR
    #c.mkdir(parents=True, exist_ok=True)
    print(f"Using Google Drive folder as BASE_DIR: {BASE_DIR}")
else:
    print(f"Using local runtime folder as BASE_DIR: {BASE_DIR}")

Mounted at /content/drive
Using Google Drive folder as BASE_DIR: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT


In [2]:

#@title Install dependencies
!pip -q install --upgrade huggingface_hub datasets pandas biopython lmdb tqdm rich


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompati

In [3]:
from pathlib import Path
from huggingface_hub import snapshot_download
import os, json, lmdb, pickle, pandas as pd
from tqdm import tqdm
from rich import print as rprint

## Generate CoT (1k Demo)

In [4]:
!pip install openai
!pip install anthropic
import asyncio
import json, os, re, tempfile, time, math
from datetime import datetime
from typing import List, Dict, Any, Iterable
from openai import OpenAI
import torch
import openai
import abc
from transformers import AutoTokenizer, AutoModelForCausalLM
import math
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from tqdm import tqdm
import anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
# Set the required environment variables.
#os.environ["ANTHROPIC_API_KEY"] =
#os.environ["MISTRAL_API_KEY"] =
os.environ["OPENAI_API_KEY"] = ""

In [6]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
#anthropic_client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
OPENAI_POLL_INTERVAL = 5
ANTHROPIC_POLL_INTERVAL = 5
OPENAI_CHUNK_SIZE = 300
ANTHROPIC_MAX_BATCH = 300

In [ ]:
BASE_DIR

PosixPath('/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT')

In [7]:
from pathlib import Path
import json

# STEP 3: Fetch & build SFT
from openai import OpenAI
client = OpenAI()

from pathlib import Path
from types import SimpleNamespace as NS

### Generate CoT with ground truth

In [ ]:
# ===================== GPT Batch: Fixed-Answer (model fills ONLY <thinking>) =====================
# The SYSTEM prompt stays as your PROTEIN_PROMPT_WITH_EXAMPLE.
# The USER message includes the exact <answer> (ground truth) and asks the model to fill ONLY <thinking>.
# -----------------------------------------------------------------------------------------------
import os, json, re, html, random, time
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# --- Required ---
os.environ.setdefault("OPENAI_API_KEY", "<PUT_YOUR_KEY_HERE>")  # or set in env beforehand
BASE_DIR         = Path("")  # ProteinDT root
MODEL            = "gpt-4o"
N_SAMPLES        = 100
TEMPERATURE      = 0.1
MAX_TOKENS       = 10000   # thinking + (copied) answer; thinking is the only part model must generate
BATCH_COMPLETION_WINDOW = "24h"
RANDOM_SEED      = 42

# --- Optional: reuse your 1k subset manifest ---
MANIFEST_PATH    = BASE_DIR / "subset_1k" / "subset_manifest_1k.csv"  # or set to None
MANIFEST_LIMIT   = 0  # 0 = no extra cap

# --- Outputs (new folder for this variant) ---
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
OUT_DIR.mkdir(parents=True, exist_ok=True)
RUN_TAG            = time.strftime("%Y%m%d-%H%M%S")
BATCH_INPUT_JSONL  = OUT_DIR / f"protein2desc_fixedans_input_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_fixedans_meta_{RUN_TAG}.json"
BATCH_INFO_JSON    = OUT_DIR / f"protein2desc_fixedans_batch_info_{RUN_TAG}.json"

client = OpenAI()

# ---------------- utils ----------------
def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0"," ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap

# ---------------- your SYSTEM prompt (unchanged) ----------------
# Use exactly your string (paste/keep your existing definition here).
def make_system_prompt_with_example(ex_uid: str, ex_seq: str, ex_gt: str) -> str:
    return (
        "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
        "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
        "### Example:\n"
        f"Protein UniProt: {ex_uid}\n"
        f"Protein sequence: {trunc(ex_seq, 1200)}\n"
        f"Answer: <answer> {ex_gt} </answer>\n\n"
        "### Now try this:\n"
        "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
        "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
        "polished 2–4 sentence description.\n\n"
        "Your final answer **must** be returned in the format:\n"
        "<thinking>\n[5 - 10 steps: Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    )

# --------------- Build messages where USER supplies the exact <answer> ---------------
def build_messages_fill_thinking_fixed_answer(system_prompt: str, uniprot_id: str, seq: str, ground_truth: str):
    """
    We give the exact <answer> block to the model and ask it to generate ONLY the <thinking> block,
    then output both blocks with the answer copied verbatim.
    """
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        #"Fill ONLY the <thinking> block using sequence-derived evidence. "
        #"Then output BOTH blocks exactly as below, keeping the <answer> text EXACTLY as provided.\n\n"
        "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n"
        f"{ground_truth}\n"
        "</answer>"
    )
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user},
    ]

# ---------------- Build & submit batch ----------------
random.seed(RANDOM_SEED)
ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# optional restriction to manifest subset
restrict_ids = None
if MANIFEST_PATH and MANIFEST_PATH.exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]

pool = restrict_ids if restrict_ids else ids[:]
random.shuffle(pool)

# Example for system prompt
ex_uid = pool[0]
ex_seq = id2seq[ex_uid]
ex_gt  = id2cap[ex_uid]
SYSTEM_PROMPT_WITH_EXAMPLE = make_system_prompt_with_example(ex_uid, ex_seq, ex_gt)

# Build items (exclude the example ID from targets)
usable = [x for x in pool if x != ex_uid]
chosen = usable[:N_SAMPLES]

meta = {}
with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
    for i, uid in enumerate(chosen):
        seq = id2seq[uid]
        gt  = id2cap[uid]  # ground truth that will be FIXED in <answer>
        cid = f"p2d_fixans_{RUN_TAG}_{i}_{uid}"
        meta[cid] = {
            "uniprot_id": uid,
            "sequence": trunc(seq, 2000),
            "ground_truth": gt,
            "example_uid": ex_uid
        }
        body = {
            "model": MODEL,
            "temperature": TEMPERATURE,
            "messages": build_messages_fill_thinking_fixed_answer(SYSTEM_PROMPT_WITH_EXAMPLE, uid, seq, gt),
            "max_tokens": MAX_TOKENS,
        }
        fout.write(json.dumps({
            "custom_id": cid,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": body
        }, ensure_ascii=False) + "\n")

BATCH_META_JSON.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"[System example used] UniProt: {ex_uid}")
print(f"Wrote batch input: {BATCH_INPUT_JSONL}")
print(f"Wrote meta map:   {BATCH_META_JSON}")

with open(BATCH_INPUT_JSONL, "rb") as f:
    input_file = client.files.create(file=f, purpose="batch")

batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window=BATCH_COMPLETION_WINDOW,
)

BATCH_INFO_JSON.write_text(json.dumps({
    "batch_id": batch.id,
    "input_file_id": input_file.id,
    "status": batch.status,
    "run_tag": RUN_TAG,
    "example_uid": ex_uid
}, indent=2), encoding="utf-8")

print("\n=== SUBMITTED BATCH ===")
print("batch_id:      ", batch.id)
print("input_file_id: ", input_file.id)
print("status:        ", batch.status)
print(f"(Saved to {BATCH_INFO_JSON})")

print("\nWhen the batch finishes, your existing fetch/merge (loose parser) cell will work as-is.")
print("It will recover the real system prompt as `instruction`, and parse <thinking>/<answer> from model outputs.")

[System example used] UniProt: A9M1F6
Wrote batch input: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_input_20250925-212446.jsonl
Wrote meta map:   /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_meta_20250925-212446.json

=== SUBMITTED BATCH ===
batch_id:       batch_68d5b32372048190b56269225499429d
input_file_id:  file-PjazxKv8wTA3dimHQM2Eco
status:         validating
(Saved to /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_batch_info_20250925-212446.json)

When the batch finishes, your existing fetch/merge (loose parser) cell will work as-is.
It will recover the real system prompt as `instruction`, and parse <thinking>/<answer> from model outputs.


In [ ]:
# ===================== Fetch output & build SFTs (Fixed-Answer variant) =====================
from pathlib import Path
import json, re
from openai import OpenAI

# Base dataset dir (same as in your submit cell)
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # <- change if needed
OUT_DIR  = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

# Auto-pick the latest run by timestamp suffix
infos = sorted(OUT_DIR.glob("protein2desc_fixedans_batch_info_20250925-211122.json"))
assert infos, f"No batch_info JSON found under {OUT_DIR}"
BATCH_INFO_JSON = infos[-1]

RUN_TAG = json.loads(BATCH_INFO_JSON.read_text())["run_tag"]
BATCH_OUTPUT_JSONL = OUT_DIR / f"protein2desc_fixedans_output_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_fixedans_meta_{RUN_TAG}.json"

# Output files
SFT_OUT_PARSED = OUT_DIR / f"protein2desc_fixedans_sft_{RUN_TAG}.json"   # parsed + answer forced to GT
SFT_OUT_RAW    = OUT_DIR / f"protein2desc_fixedans_raw_{RUN_TAG}.json"   # raw assistant text

client = OpenAI()

# ---- helpers ----
def fetch_batch_output_if_ready(batch_id: str, out_jsonl: Path):
    b = client.batches.retrieve(batch_id)
    if not getattr(b, "output_file_id", None):
        return False, str(b.status)
    content = client.files.content(b.output_file_id)
    data = content.read()
    out_jsonl.write_bytes(data)
    return True, out_jsonl

def extract_assistant_content(obj) -> str:
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_system_message(obj) -> str:
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    for m in msgs:
        if m.get("role") == "system":
            return m.get("content", "")
    return ""

# Loose tag extractors (handle missing closing tags)
def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    if not text:
        return "", ""
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text[th_end:] if th_found else text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    if not th_found and an_found:
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

def normalize_text(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    return s

# ---- run ----
binfo = json.loads(BATCH_INFO_JSON.read_text())
batch_id = binfo["batch_id"]

ok, res = fetch_batch_output_if_ready(batch_id, BATCH_OUTPUT_JSONL)
if not ok:
    print(f"Batch not complete yet: status={res}  (batch_id={batch_id})")
else:
    meta = json.loads(BATCH_META_JSON.read_text(encoding="utf-8"))

    parsed_records = []
    raw_records    = []
    kept = 0
    altered_answers = 0
    missing_thinking_tag = 0
    one_tag_answer = 0

    with open(BATCH_OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj.get("error"):
                continue

            content = extract_assistant_content(obj) or ""
            system_prompt = extract_system_message(obj)
            cid  = obj.get("custom_id")
            mrow = meta.get(cid, {})
            uid, seq, gt = mrow.get("uniprot_id"), mrow.get("sequence"), mrow.get("ground_truth")

            # RAW record (no parsing/overrides)
            raw_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": content,
                "meta": {"ground_truth_caption": gt, "custom_id": cid}
            })

            # PARSED record (loose parse; force answer to GT)
            thinking, model_answer = parse_thinking_answer_loose(content)
            if "<thinking" not in content.lower():
                missing_thinking_tag += 1
            if "<answer" in content.lower() and "</answer>" not in content.lower():
                one_tag_answer += 1

            # Compare model's answer to GT (normalize spaces); then FORCE to GT
            model_eq_gt = normalize_text(model_answer) == normalize_text(gt)
            if not model_eq_gt:
                altered_answers += 1

            parsed_records.append({
                "instruction": system_prompt,   # real system prompt as instruction
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": (
                    f"<thinking>\n{thinking}\n</thinking>\n\n"
                    f"<answer>\n{gt}\n</answer>"      # <-- force canonical GT answer
                ),
                "meta": {
                    "ground_truth_caption": gt,
                    "custom_id": cid,
                    "model_answer_equal_to_gt": model_eq_gt
                }
            })
            kept += 1

    # Save both
    SFT_OUT_PARSED.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
    SFT_OUT_RAW.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

    print({
        "run_tag": RUN_TAG,
        "kept": kept,
        "parsed_out": str(SFT_OUT_PARSED),
        "raw_out": str(SFT_OUT_RAW),
        "missing_thinking_tag": missing_thinking_tag,
        "one_tag_answer": one_tag_answer,
        "answer_altered_by_model": altered_answers
    })

    # Quick preview
    if parsed_records:
        print("\n--- SAMPLE (parsed) ---")
        print("INSTRUCTION (truncated):", parsed_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", parsed_records[0]["output"][:300], "...")
    if raw_records:
        print("\n--- SAMPLE (raw) ---")
        print("INSTRUCTION (truncated):", raw_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", raw_records[0]["output"][:300], "...")

{'run_tag': '20250925-211122', 'kept': 100, 'parsed_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-211122.json', 'raw_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_raw_20250925-211122.json', 'missing_thinking_tag': 0, 'one_tag_answer': 32, 'answer_altered_by_model': 89}

--- SAMPLE (parsed) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. The sequence begins with "MSEIRKNDHRLMQVLLAPVISEKATLVADKNEQVVFEVAPDATKQEVKAAVELLFKVEVDSVNVLVQKGKQKRFGRSMGRRKDVKKAYVCLKPGQEINFEAEAK", which suggests a lack of a signal peptide, indicating a cytoplasmic localization.
2. The sequence is relatively short, suggesting it might be part of a l ...

--- SAMPLE (raw) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. The sequence begins with "MSEIRKNDHRLMQVLLAPVISEKATLVADKNEQVVFEVAPDATKQEVKAAVELLFKVEVDS

In [ ]:
# ===================== Compare CoT Lengths across ALL versions/tags =====================
from pathlib import Path
import json, re
import pandas as pd

# ---- base path (edit if needed) ----
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")

V1_DIR     = BASE_DIR / "gpt_batch_protein2desc"
FIXED_DIR  = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

# ---- where to save results ----
OUT_PER_ITEM_CSV = BASE_DIR / "cot_length_comparison_all_versions.csv"
OUT_SUMMARY_CSV  = BASE_DIR / "cot_length_summary_all_versions.csv"

# ---- loose tag extraction (handles missing closers) ----
def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        # no closing tag → capture to end
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def extract_thinking(text: str) -> str:
    thinking, _, found = _extract_tag_loose(text or "", "thinking")
    if found:
        return thinking
    # fallback: treat whole text as "thinking"
    return (text or "").strip().rstrip('", \t\r\n')

def load_sft(path: Path):
    data = json.loads(path.read_text(encoding="utf-8"))
    if not isinstance(data, list):
        raise ValueError(f"Expected list at {path}")
    return data

def lengths_for(records, set_label: str, tag: str, src_file: Path):
    rows = []
    for i, r in enumerate(records):
        out = (r.get("output") or "")
        t = extract_thinking(out)
        rows.append({
            "family": set_label,                          # 'v1_free' or 'fixed_answer'
            "tag": tag,                                   # e.g. timestamp or 'default'
            "set": f"{set_label}:{tag}",                  # combined label
            "idx": i,
            "uniprot_id": ((r.get("input") or {}).get("uniprot_id") or ""),
            "char_len": len(t),
            "word_len": len(t.split()),
            "empty": (len(t) == 0),
            "source_file": str(src_file)
        })
    return rows

def summarize_group(df, set_label: str):
    sub = df[df["set"] == set_label]
    n = len(sub)
    n_empty = int(sub["empty"].sum())
    non_empty = sub[~sub["empty"]]
    def stats_series(series):
        if len(series) == 0:
            return {"mean": 0, "median": 0, "p10": 0, "p90": 0, "min": 0, "max": 0}
        return {
            "mean": float(series.mean()),
            "median": float(series.median()),
            "p10": float(series.quantile(0.10)),
            "p90": float(series.quantile(0.90)),
            "min": int(series.min()),
            "max": int(series.max()),
        }
    ch_all = stats_series(sub["char_len"])
    wd_all = stats_series(sub["word_len"])
    ch_ne  = stats_series(non_empty["char_len"])
    wd_ne  = stats_series(non_empty["word_len"])
    return {
        "set": set_label,
        "count": n,
        "empty_count": n_empty,
        "empty_pct": round(100 * n_empty / n, 2) if n else 0.0,
        "chars_all_mean": round(ch_all["mean"], 2),
        "chars_all_median": ch_all["median"],
        "chars_nonempty_mean": round(ch_ne["mean"], 2),
        "chars_nonempty_median": ch_ne["median"],
        "words_all_mean": round(wd_all["mean"], 2),
        "words_all_median": wd_all["median"],
        "words_nonempty_mean": round(wd_ne["mean"], 2),
        "words_nonempty_median": wd_ne["median"],
        "chars_all_p10": ch_all["p10"],
        "chars_all_p90": ch_all["p90"],
        "words_all_p10": wd_all["p10"],
        "words_all_p90": wd_all["p90"],
        "chars_all_min": ch_all["min"],
        "chars_all_max": ch_all["max"],
        "words_all_min": wd_all["min"],
        "words_all_max": wd_all["max"],
    }

# ---- discover all SFT files & tags ----
found_files = []

# v1 variants:
# - protein2desc_cot_sft.json                → tag = 'default'
# - protein2desc_cot_sft_<TAG>.json          → tag = <TAG>
if V1_DIR.exists():
    for p in V1_DIR.glob("protein2desc_cot_sft.json"):
        found_files.append(("v1_free", "default", p))
    for p in V1_DIR.glob("protein2desc_cot_sft_*.json"):
        # capture tag after 'sft_'
        m = re.match(r"protein2desc_cot_sft_(?P<tag>[^.]+)\.json$", p.name)
        tag = m.group("tag") if m else p.stem.replace("protein2desc_cot_sft_", "")
        found_files.append(("v1_free", tag, p))

# fixed-answer variants:
# - protein2desc_fixedans_sft_<TAG>.json     → tag = <TAG>
if FIXED_DIR.exists():
    for p in FIXED_DIR.glob("protein2desc_fixedans_sft_*.json"):
        m = re.match(r"protein2desc_fixedans_sft_(?P<tag>[^.]+)\.json$", p.name)
        if not m:
            continue
        tag = m.group("tag")
        found_files.append(("fixed_answer", tag, p))

if not found_files:
    raise FileNotFoundError("No SFT files found in either folder.")

print("Discovered versions:")
for fam, tag, p in found_files:
    print(f" - [{fam}:{tag}]  {p}")

# ---- load all, compute per-item rows ----
rows = []
for fam, tag, p in found_files:
    try:
        recs = load_sft(p)
        rows += lengths_for(recs, set_label=fam, tag=tag, src_file=p)
    except Exception as e:
        print(f"[WARN] Skipping {p}: {e}")

df = pd.DataFrame(rows)
assert not df.empty, "No records loaded from discovered SFT files."

# ---- build summary for each version (set) ----
sets = sorted(df["set"].unique())
summary_rows = [summarize_group(df, s) for s in sets]
summary_df = pd.DataFrame(summary_rows).sort_values(["set"]).reset_index(drop=True)

print("\n=== CoT Length Summary by Version ===")
print(summary_df.to_string(index=False))

# ---- save outputs ----
OUT_PER_ITEM_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_PER_ITEM_CSV, index=False)
summary_df.to_csv(OUT_SUMMARY_CSV, index=False)

print("\nSaved per-item lengths to:", OUT_PER_ITEM_CSV)
print("Saved summary to:", OUT_SUMMARY_CSV)

Discovered versions:
 - [v1_free:default]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json
 - [v1_free:run_b0778017]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft_run_b0778017.json
 - [fixed_answer:20250925-180228]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-180228.json
 - [fixed_answer:20250925-204511]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-204511.json
 - [fixed_answer:20250925-212446]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-212446.json
 - [fixed_answer:20250925-211122]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-211122.json

=== 

In [ ]:
# moving files if you set wrong working directory.
import os, shutil, hashlib
from pathlib import Path


SRC  = Path("/content/gpt_batch_protein2desc")
DEST = BASE_DIR / "gpt_batch_protein2desc"

def count_files_and_bytes(root: Path):
    n, b = 0, 0
    for p in root.rglob("*"):
        if p.is_file():
            n += 1
            try:
                b += p.stat().st_size
            except Exception:
                pass
    return n, b

if not SRC.exists():
    raise FileNotFoundError(f"Source folder not found: {SRC}")

DEST.mkdir(parents=True, exist_ok=True)

src_files, src_bytes = count_files_and_bytes(SRC)
print(f"[SRC] {SRC}  files={src_files}  size={src_bytes/1e6:.2f} MB")

if not any(DEST.iterdir()):

    for item in SRC.iterdir():
        shutil.move(str(item), str(DEST))
else:

    for root, dirs, files in os.walk(SRC):
        rel = Path(root).relative_to(SRC)
        dest_dir = DEST / rel
        dest_dir.mkdir(parents=True, exist_ok=True)
        for d in dirs:
            (dest_dir / d).mkdir(parents=True, exist_ok=True)
        for f in files:
            s = Path(root) / f
            t = dest_dir / f
            if t.exists():
                try:
                    t.unlink()
                except IsADirectoryError:
                    shutil.rmtree(t)
            try:
                shutil.move(str(s), str(t))
            except Exception:
                shutil.copy2(str(s), str(t))
                s.unlink()

dest_files, dest_bytes = count_files_and_bytes(DEST)
print(f"[DEST] {DEST}  files={dest_files}  size={dest_bytes/1e6:.2f} MB")

if dest_files >= src_files and dest_bytes >= src_bytes * 0.999:
    try:
        shutil.rmtree(SRC)
        print(f"[OK] Moved and removed source: {SRC}")
    except Exception as e:
        print(f"[WARN] Could not remove source (remove manually if needed): {e}")
else:
    raise RuntimeError("Sanity check failed: destination counts smaller than source; aborting deletion.")

important = [
    "protein2desc_input.jsonl",
    "protein2desc_meta.json",
    "protein2desc_batch_info.json",
    "protein2desc_output.jsonl",
    "protein2desc_cot_sft.json",
]
print("\nKey files now under:", DEST)
for name in important:
    p = DEST / name
    print(" -", p, ("[OK]" if p.exists() else "[missing]"))

[SRC] /content/gpt_batch_protein2desc  files=5  size=9.33 MB
[DEST] /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc  files=5  size=9.33 MB
[OK] Moved and removed source: /content/gpt_batch_protein2desc

Key files now under: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_input.jsonl [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_meta.json [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_batch_info.json [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_output.jsonl [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json [OK]


## Large Scale CoT generation

In [8]:
# ===================== Fixed-Answer batches with pause/resume (50k split; submit N items per run) =====================
import os, json, re, html, random, time, math
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# ---------- CONFIG ----------
BASE_DIR          = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # change if needed
OUT_DIR           = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Model/Batch
MODEL             = "gpt-4o"
TEMPERATURE       = 0.1
MAX_TOKENS        = 4000
COMPLETION_WINDOW = "24h"          # "24h" or "48h"
SUBMIT_SLEEP_SEC  = 0.0            # sleep between chunk submissions if desired

# Dataset size & chunking
N_TOTAL           = 541159         # total target items
CHUNK_SIZE        = 1_000          # items per batch/chunk
ITEMS_PER_RUN     = 5000        # how many to submit THIS run (e.g., 10k now, 10k later)
SHUFFLE_SEED      = 42

# Optional: restrict pool via a CSV manifest with "uniprot_id"
MANIFEST_PATH     = None           # e.g., BASE_DIR/"subset_50k/manifest_50k.csv"
MANIFEST_LIMIT    = 0              # 0 = no extra cap

client = OpenAI()

# ---------- Helpers ----------
def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0", " ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    """SwissProtCLAP: alternating lines: id then value."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap


# ---------------- your SYSTEM prompt (unchanged) ----------------
# Use exactly your string (paste/keep your existing definition here).
def make_system_prompt_with_example(ex_uid: str, ex_seq: str, ex_gt: str) -> str:
    return (
        "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
        "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
        "### Example:\n"
        f"Protein UniProt: {ex_uid}\n"
        f"Protein sequence: {trunc(ex_seq, 1200)}\n"
        f"Answer: <answer> {ex_gt} </answer>\n\n"
        "### Now try this:\n"
        "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
        "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
        "polished 2–4 sentence description.\n\n"
        "Your final answer **must** be returned in the format:\n"
        "<thinking>\n[steps: Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    )

# --------------- Build messages where USER supplies the exact <answer> ---------------
def build_messages_fill_thinking_fixed_answer(system_prompt: str, uniprot_id: str, seq: str, ground_truth: str):
    """
    We give the exact <answer> block to the model and ask it to generate ONLY the <thinking> block,
    then output both blocks with the answer copied verbatim.
    """
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        #"Fill ONLY the <thinking> block using sequence-derived evidence. "
        #"Then output BOTH blocks exactly as below, keeping the <answer> text EXACTLY as provided.\n\n"
        "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n"
        f"{ground_truth}\n"
        "</answer>"
    )
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user},
    ]


def write_json(path: Path, obj: dict|list):
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")

def parse_chunk_index_from_info(path: Path) -> int|None:
    """Extract chunk index from our naming pattern: ..._chunkXXX.json"""
    m = re.search(r"_chunk(\d{3})\.json$", path.name)
    return int(m.group(1)) if m else None

# ---------- Prepare or load stable order ----------
ORDER_JSON = OUT_DIR / "protein2desc_fixedans_idorder.json"

ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# optional manifest restriction
if MANIFEST_PATH and Path(MANIFEST_PATH).exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]
    pool_initial = [i for i in restrict_ids if i in id2seq and i in id2cap]
else:
    pool_initial = ids[:]

if ORDER_JSON.exists():
    # Resume mode: load stable order + example
    order_info = json.loads(ORDER_JSON.read_text(encoding="utf-8"))
    order_ids  = order_info["order_ids"]
    ex_uid     = order_info["system_example"]["ex_uid"]
    ex_seq     = order_info["system_example"]["ex_seq_trunc"]
    ex_gt      = order_info["system_example"]["ex_gt"]
    saved_seed = order_info.get("seed", None)
    saved_total = order_info.get("n_total_planned", None)
    if saved_total and saved_total != N_TOTAL:
        print(f"[NOTE] Existing order planned {saved_total}; current N_TOTAL={N_TOTAL}. Proceeding with saved order length={len(order_ids)}.")
    print(f"[RESUME] Loaded order file with {len(order_ids)} ids; ex_uid={ex_uid}")
else:
    # First run: create deterministic order & pick example
    random.seed(SHUFFLE_SEED)
    pool = pool_initial[:]
    random.shuffle(pool)
    # Pick example used in system prompt
    ex_uid = pool[0]
    ex_seq = id2seq[ex_uid]
    ex_gt  = id2cap[ex_uid]
    # Build the target order excluding example
    usable = [x for x in pool if x != ex_uid]
    if len(usable) < N_TOTAL:
        print(f"[WARN] Pool smaller than requested N_TOTAL={N_TOTAL}. Using {len(usable)}.")
    order_ids = usable[:N_TOTAL]

    order_info = {
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "seed": SHUFFLE_SEED,
        "n_total_planned": len(order_ids),
        "chunk_size": CHUNK_SIZE,
        "system_example": {
            "ex_uid": ex_uid,
            "ex_seq_trunc": trunc(ex_seq, 1200),
            "ex_gt": ex_gt
        },
        "order_ids": order_ids
    }
    write_json(ORDER_JSON, order_info)
    print(f"[INIT] Wrote stable order: {ORDER_JSON.name}  |  ids={len(order_ids)}  |  ex_uid={ex_uid}")

# ---------- Determine which chunks already submitted ----------
info_files = sorted(OUT_DIR.glob("protein2desc_fixedans_batch_info_*_chunk???.json"))
submitted_chunks = set()
for p in info_files:
    idx = parse_chunk_index_from_info(p)
    if idx is not None:
        submitted_chunks.add(idx)

total_chunks = math.ceil(len(order_ids) / CHUNK_SIZE)
remaining_chunks = [i for i in range(total_chunks) if i not in submitted_chunks]

remaining_items = sum(
    min(CHUNK_SIZE, len(order_ids) - (i * CHUNK_SIZE))
    for i in remaining_chunks
)

# How many items to submit this run?
to_submit_items = min(ITEMS_PER_RUN, remaining_items)
to_submit_chunks = []
items_counted = 0
for ci in remaining_chunks:
    chunk_start = ci * CHUNK_SIZE
    chunk_end   = min((ci+1)*CHUNK_SIZE, len(order_ids))
    sz = chunk_end - chunk_start
    if items_counted + sz <= to_submit_items:
        to_submit_chunks.append(ci)
        items_counted += sz
    else:
        # partial chunk not supported: we submit full chunks; stop here
        break

print(f"\n[PROGRESS] total_ids={len(order_ids)}  chunk_size={CHUNK_SIZE}  total_chunks={total_chunks}")
print(f"  already_submitted_chunks={len(submitted_chunks)}  remaining_chunks={len(remaining_chunks)}")
print(f"  plan_this_run: submit_items={to_submit_items}  =>  chunks={len(to_submit_chunks)} -> {to_submit_chunks[:6]}{'...' if len(to_submit_chunks)>6 else ''}")

if not to_submit_chunks:
    print("\nNothing to submit. All planned chunks are already submitted (or ITEMS_PER_RUN=0).")
else:
    # Build system prompt (use stored example)
    SYSTEM_PROMPT = make_system_prompt_with_example(ex_uid, ex_seq, ex_gt)

    # ---------- Submit each selected chunk ----------
    for c in to_submit_chunks:
        chunk_start = c * CHUNK_SIZE
        chunk_end   = min((c+1) * CHUNK_SIZE, len(order_ids))
        chunk_ids   = order_ids[chunk_start:chunk_end]
        chunk_tag   = time.strftime("%Y%m%d-%H%M%S")
        chunk_pad   = f"{c:03d}"

        # file names for this chunk
        BATCH_INPUT_JSONL = OUT_DIR / f"protein2desc_fixedans_input_{chunk_tag}_chunk{chunk_pad}.jsonl"
        BATCH_META_JSON   = OUT_DIR / f"protein2desc_fixedans_meta_{chunk_tag}_chunk{chunk_pad}.json"
        BATCH_INFO_JSON   = OUT_DIR / f"protein2desc_fixedans_batch_info_{chunk_tag}_chunk{chunk_pad}.json"

        # assemble JSONL + meta
        meta = {}
        with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
            for i, uid in enumerate(chunk_ids):
                seq = id2seq[uid]
                gt  = id2cap[uid]  # ground truth answer
                cid = f"p2d_fixans_{chunk_tag}_c{chunk_pad}_{i:04d}_{uid}"
                meta[cid] = {
                    "uniprot_id": uid,
                    "sequence": trunc(seq, 2000),
                    "ground_truth": gt,
                    "example_uid": ex_uid,
                    "chunk_index": c,
                    "chunk_size": len(chunk_ids),
                    "order_start": chunk_start,
                    "order_end": chunk_end
                }
                body = {
                    "model": MODEL,
                    "temperature": TEMPERATURE,
                    "messages": build_messages_fill_thinking_fixed_answer(SYSTEM_PROMPT, uid, seq, gt),
                    "max_tokens": MAX_TOKENS,
                }
                fout.write(json.dumps({
                    "custom_id": cid,
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": body
                }, ensure_ascii=False) + "\n")

        write_json(BATCH_META_JSON, meta)

        # submit the batch
        with open(BATCH_INPUT_JSONL, "rb") as f:
            input_file = client.files.create(file=f, purpose="batch")

        batch = client.batches.create(
            input_file_id=input_file.id,
            endpoint="/v1/chat/completions",
            completion_window=COMPLETION_WINDOW,
        )

        write_json(BATCH_INFO_JSON, {
            "batch_id": batch.id,
            "input_file_id": input_file.id,
            "status": batch.status,
            "run_tag": chunk_tag,
            "chunk_index": c,
            "chunk_size": len(chunk_ids),
            "n_total_planned": len(order_ids),
            "order_range": [chunk_start, chunk_end],
            "file_prefix": f"protein2desc_fixedans_*_{chunk_tag}_chunk{chunk_pad}.*"
        })

        print(f"[SUBMITTED] chunk={c:03d} items={len(chunk_ids):4d} batch_id={batch.id} status={batch.status}")
        if SUBMIT_SLEEP_SEC > 0:
            time.sleep(SUBMIT_SLEEP_SEC)

    print("\nDone submitting this run.\nTip: re-run this cell later to submit the next 10k (or adjust ITEMS_PER_RUN).")

[NOTE] Existing order planned 541158; current N_TOTAL=541159. Proceeding with saved order length=541158.
[RESUME] Loaded order file with 541158 ids; ex_uid=Q2LSF9

[PROGRESS] total_ids=541158  chunk_size=1000  total_chunks=542
  already_submitted_chunks=5  remaining_chunks=537
  plan_this_run: submit_items=5000  =>  chunks=5 -> [5, 6, 7, 8, 9]
[SUBMITTED] chunk=005 items=1000 batch_id=batch_68dd98823ca08190aceab6a40d72f912 status=validating
[SUBMITTED] chunk=006 items=1000 batch_id=batch_68dd9883a0d881908455cbefc8245f4f status=validating
[SUBMITTED] chunk=007 items=1000 batch_id=batch_68dd9884f1308190bd9275dadf5ed177 status=validating
[SUBMITTED] chunk=008 items=1000 batch_id=batch_68dd9886813081909f1d6cd1b97ecfc4 status=validating
[SUBMITTED] chunk=009 items=1000 batch_id=batch_68dd9888a4908190a107f824c63b6eda status=validating

Done submitting this run.
Tip: re-run this cell later to submit the next 10k (or adjust ITEMS_PER_RUN).


In [ ]:
# === Verify SwissProtCLAP has 541,159 sequences ===
from pathlib import Path

# If BASE_DIR is not already defined in your notebook, set it here:
try:
    BASE_DIR
except NameError:
    BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")

sp = BASE_DIR / "SwissProtCLAP"
seq_fp = sp / "protein_sequence.txt"
txt_fp = sp / "text_sequence.txt"
EXPECTED = 541_159

def count_two_line_records(p: Path) -> tuple[int, int, bool]:
    """Return (records, total_lines, is_even). Each record is 2 lines: ID then value."""
    if not p.exists():
        raise FileNotFoundError(f"Missing file: {p}")
    total = 0
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        for total, _ in enumerate(f, 1):
            pass
    return total // 2, total, (total % 2 == 0)

seq_n, seq_lines, seq_even = count_two_line_records(seq_fp)
txt_n, txt_lines, txt_even = count_two_line_records(txt_fp)

print(f"[protein_sequence.txt] lines={seq_lines:,}  records={seq_n:,}  even_lines={seq_even}")
print(f"[text_sequence.txt]    lines={txt_lines:,}  records={txt_n:,}  even_lines={txt_even}")

ok = (seq_even and txt_even and seq_n == txt_n == EXPECTED)
print("\nOK:" if ok else "\nMISMATCH:",
      f"SwissProtCLAP contains {seq_n:,} sequence-caption pairs (expected {EXPECTED:,}).")

[protein_sequence.txt] lines=1,082,318  records=541,159  even_lines=True
[text_sequence.txt]    lines=1,082,318  records=541,159  even_lines=True

OK: SwissProtCLAP contains 541,159 sequence-caption pairs (expected 541,159).


In [ ]:
# ===================== Fetch ALL submitted fixed-answer batches into subfolders =====================
from pathlib import Path
import json, re
from openai import OpenAI

# --- Config ---
BASE_DIR  = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # change if needed
OUT_DIR   = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

RESULTS_ROOT = OUT_DIR / "sft_results"   # per-chunk subfolders land here
RESULTS_ROOT.mkdir(parents=True, exist_ok=True)

ONLY_COMPLETED     = True    # True: fetch only completed; False: attempt fetch if output_file_id exists anyway
FORCE_REDOWNLOAD   = False   # True: always re-download and rebuild even if files exist
BUILD_SFT_FILES    = True    # build sft_parsed.json and sft_raw.json
PRINT_SAMPLE       = True    # show one parsed & raw sample per run

client = OpenAI()

# --- Helpers: parsing / extraction ---
def fetch_batch_output_if_ready(batch_id: str) -> tuple[bool, bytes|str]:
    """Return (ok, data-or-status). ok=True & data=bytes of jsonl if downloadable; else ok=False & status str."""
    b = client.batches.retrieve(batch_id)
    status = getattr(b, "status", "unknown")
    if ONLY_COMPLETED and status != "completed":
        return False, status
    if not getattr(b, "output_file_id", None):
        return False, status
    content = client.files.content(b.output_file_id)
    return True, content.read()

def extract_assistant_content(obj) -> str:
    """Batch line → response.body.choices[0].message.content"""
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_system_message(obj) -> str:
    """Batch line → request.body.messages[*] (system)"""
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    for m in msgs:
        if m.get("role") == "system":
            return m.get("content", "")
    return ""

def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    """Tolerant CoT parser: accepts missing closing tags and open-only <answer>."""
    if not text:
        return "", ""
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text[th_end:] if th_found else text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    if not th_found and an_found:
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

def normalize_text(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    return s

def parse_chunk_parts(fname: str):
    """
    From 'protein2desc_fixedans_batch_info_<run_tag>_chunkXYZ.json' return (run_tag, chunk_pad).
    """
    m = re.match(r"protein2desc_fixedans_batch_info_(?P<tag>.+?)_chunk(?P<pad>\d{3})\.json$", fname)
    if not m:
        return None, None
    return m.group("tag"), m.group("pad")

# --- Discover all batch_info files (any run) ---
info_files = sorted(OUT_DIR.glob("protein2desc_fixedans_batch_info_*_chunk???.json"))
if not info_files:
    raise FileNotFoundError(f"No batch_info files found under {OUT_DIR}")

print(f"Discovered {len(info_files)} submitted chunks.")

# --- Process each chunk ---
global_kept = 0
global_altered_answers = 0
global_missing_thinking = 0
global_one_tag_answer = 0
fetched = 0
skipped = 0
waiting = 0

for info_path in info_files:
    run_tag, chunk_pad = parse_chunk_parts(info_path.name)
    if not run_tag:
        print(f"[WARN] Skip unparsable info file name: {info_path.name}")
        continue

    # per-chunk file names
    meta_path = OUT_DIR / f"protein2desc_fixedans_meta_{run_tag}_chunk{chunk_pad}.json"
    out_dir   = RESULTS_ROOT / f"{run_tag}_chunk{chunk_pad}"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_jsonl_path = out_dir / "output.jsonl"
    parsed_path    = out_dir / "sft_parsed.json"
    raw_path       = out_dir / "sft_raw.json"

    # load batch info
    binfo = json.loads(info_path.read_text(encoding="utf-8"))
    batch_id = binfo["batch_id"]

    # fast-skip if everything already there (and not forcing)
    if (out_jsonl_path.exists() and parsed_path.exists() and raw_path.exists()
        and meta_path.exists() and not FORCE_REDOWNLOAD):
        skipped += 1
        continue

    ok, data_or_status = fetch_batch_output_if_ready(batch_id)
    if not ok:
        waiting += 1
        print(f"[WAIT] {info_path.name}  status={data_or_status}")
        continue

    # write downloaded output
    out_jsonl_path.write_bytes(data_or_status)

    # copy meta & info to subfolder for convenience
    if meta_path.exists():
        (out_dir / meta_path.name).write_text(meta_path.read_text(encoding="utf-8"), encoding="utf-8")
    (out_dir / info_path.name).write_text(info_path.read_text(encoding="utf-8"), encoding="utf-8")

    # Build SFTs (parsed + raw)
    if BUILD_SFT_FILES:
        assert meta_path.exists(), f"Missing meta for chunk: {meta_path}"
        meta = json.loads(meta_path.read_text(encoding="utf-8"))

        parsed_records = []
        raw_records    = []
        kept = 0
        altered_answers = 0
        missing_thinking_tag = 0
        one_tag_answer = 0

        with open(out_jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                line=line.strip()
                if not line:
                    continue
                obj = json.loads(line)
                if obj.get("error"):
                    continue

                content = extract_assistant_content(obj) or ""
                system_prompt = extract_system_message(obj)
                cid  = obj.get("custom_id")
                mrow = meta.get(cid, {})
                uid, seq, gt = mrow.get("uniprot_id"), mrow.get("sequence"), mrow.get("ground_truth")

                # Raw copy
                raw_records.append({
                    "instruction": system_prompt,
                    "input": {"uniprot_id": uid, "protein_sequence": seq},
                    "output": content,
                    "meta": {"ground_truth_caption": gt, "custom_id": cid, "run_tag": run_tag, "chunk": int(chunk_pad)}
                })

                # Parsed, force answer to GT
                thinking, model_answer = parse_thinking_answer_loose(content)
                if "<thinking" not in content.lower():
                    missing_thinking_tag += 1
                if "<answer" in content.lower() and "</answer>" not in content.lower():
                    one_tag_answer += 1
                model_eq_gt = normalize_text(model_answer) == normalize_text(gt)
                if not model_eq_gt:
                    altered_answers += 1

                parsed_records.append({
                    "instruction": system_prompt,
                    "input": {"uniprot_id": uid, "protein_sequence": seq},
                    "output": f"<thinking>\n{thinking}\n</thinking>\n\n<answer>\n{gt}\n</answer>",
                    "meta": {
                        "ground_truth_caption": gt,
                        "custom_id": cid,
                        "run_tag": run_tag,
                        "chunk": int(chunk_pad),
                        "model_answer_equal_to_gt": model_eq_gt
                    }
                })
                kept += 1

        parsed_path.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
        raw_path.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

        # per-chunk summary
        print({
            "chunk": int(chunk_pad),
            "run_tag": run_tag,
            "kept": kept,
            "altered_answers": altered_answers,
            "missing_thinking_tag": missing_thinking_tag,
            "one_tag_answer": one_tag_answer,
            "dir": str(out_dir)
        })

        # global rollup
        global_kept += kept
        global_altered_answers += altered_answers
        global_missing_thinking += missing_thinking_tag
        global_one_tag_answer += one_tag_answer

        # Optional sample print
        if PRINT_SAMPLE and parsed_records:
            print("\n--- SAMPLE (parsed) ---")
            print("INSTR (trunc):", parsed_records[0]["instruction"][:200], "…")
            print("OUTPUT (first 240 chars):", parsed_records[0]["output"][:240], "…")
            print("--- SAMPLE (raw) ---")
            print("OUTPUT (first 240 chars):", raw_records[0]["output"][:240], "…")
            PRINT_SAMPLE = False  # only once

    fetched += 1

# --- final report ---
print("\n=== FETCH SUMMARY ===")
print({
    "chunks_discovered": len(info_files),
    "chunks_fetched_this_run": fetched,
    "chunks_already_done_skipped": skipped,
    "chunks_waiting_not_ready": waiting,
    "records_kept_total": global_kept,
    "answer_altered_total": global_altered_answers,
    "missing_thinking_tag_total": global_missing_thinking,
    "one_tag_answer_total": global_one_tag_answer,
    "results_root": str(RESULTS_ROOT)
})

Discovered 5 submitted chunks.
{'chunk': 0, 'run_tag': '20250930-174241', 'kept': 980, 'altered_answers': 848, 'missing_thinking_tag': 0, 'one_tag_answer': 76, 'dir': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/sft_results/20250930-174241_chunk000'}

--- SAMPLE (parsed) ---
INSTR (trunc):  …
OUTPUT (first 240 chars): <thinking>
The sequence is relatively short and contains a high proportion of cysteine (C) and histidine (H) residues, which are often involved in metal ion coordination, suggesting a potential zinc finger motif. The presence of multiple ar …
--- SAMPLE (raw) ---
OUTPUT (first 240 chars): <thinking>
The sequence is relatively short and contains a high proportion of cysteine (C) and histidine (H) residues, which are often involved in metal ion coordination, suggesting a potential zinc finger motif. The presence of multiple ar …
{'chunk': 1, 'run_tag': '20250930-174245', 'kept': 996, 'altered_answers': 868, 'missing_thinking_

## Generate 3Di Tokens

In [ ]:
# === PDB/mmCIF -> 3Di JSONL builder (Python path via mini3di) ===
# Installs (run once per runtime)
!pip -q install mini3di biopython pandas tqdm

import json, re, html, sys
from pathlib import Path
from typing import Dict, Optional, Tuple, List
import pandas as pd
from tqdm import tqdm

# ---------- Config ----------
BASE_DIR       = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")
SWISSPROT_DIR  = BASE_DIR / "SwissProtCLAP"                    # expects protein_sequence.txt / text_sequence.txt
MANIFEST_CSV   = BASE_DIR / "subset_1k" / "subset_manifest_1k.csv"  # has uniprot_id column (and maybe caption_preview etc.)
DOWNLOAD_DIR   = BASE_DIR / "downloads"                        # where you stored AF2/experimental files per UniProt (from earlier steps)
OUT_JSONL      = BASE_DIR / "protein2desc_3di.jsonl"          # output file
PROMPT_TMPL    = ("You are a professional protein biologist. Based only on the amino-acid sequence, "
                  "write a natural, concise, biologically accurate description of the protein.\n\n"
                  "Protein sequence:\n{seq}\n")
# If you instead want to use the exact prompt you used for SFT: replace PROMPT_TMPL with that string.

# ---------- Load SwissProtCLAP sequences & captions ----------
def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line:
                break
            val_line = f.readline()
            if not val_line:
                break
            yield id_line.strip(), val_line.strip()

def strip_html(s: str) -> str:
    s = html.unescape(s or "")
    s = re.sub(r"<[^>]+>", " ", s)
    return re.sub(r"\s+", " ", s).strip()

seq_fp = SWISSPROT_DIR / "protein_sequence.txt"
txt_fp = SWISSPROT_DIR / "text_sequence.txt"
assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP at {SWISSPROT_DIR}"

id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}

# ---------- Load manifest (which UniProt IDs to include) ----------
dfm = pd.read_csv(MANIFEST_CSV)
if "uniprot_id" not in dfm.columns:
    # older manifest may use 'uniprot'
    if "uniprot" in dfm.columns:
        dfm = dfm.rename(columns={"uniprot": "uniprot_id"})
    else:
        raise ValueError("Manifest must include a 'uniprot_id' column.")

uids = [u for u in dfm["uniprot_id"].astype(str).tolist() if u in id2seq]

# ---------- mini3di encoder + parsers ----------
import mini3di
from Bio.PDB import PDBParser, MMCIFParser, PPBuilder
from Bio.PDB.Polypeptide import three_to_one

ENCODER = mini3di.Encoder()
PDB_PARSER   = PDBParser(QUIET=True)
MMCIF_PARSER = MMCIFParser(QUIET=True)

def structure_seq_from_chain(chain) -> str:
    """Extract one-letter sequence from a Bio.PDB chain (ignoring non-standard residues)."""
    seq = []
    for res in chain:
        if res.id[0] != " ":  # skip hetero/waters
            continue
        if "CA" not in res:
            continue
        resname = res.get_resname().strip()
        try:
            aa = three_to_one(resname)
        except KeyError:
            continue
        seq.append(aa)
    return "".join(seq)

def encode_chain_to_3di(chain) -> str:
    """Get 3Di string for a chain using mini3di."""
    states = ENCODER.encode_chain(chain)
    seq3di = ENCODER.build_sequence(states)
    return str(seq3di)

def pick_best_chain_for_uid(structure, target_len: int) -> Optional[Tuple[str, str, int]]:
    """
    Heuristic: pick the chain whose extracted AA length is (a) longest and
    (b) within 40–120% of the target length (to avoid weird ligands/short peptides).
    Returns (chain_id, aa_seq_from_structure, length) or None.
    """
    best = None
    for chain in structure.get_chains():
        aaseq = structure_seq_from_chain(chain)
        L = len(aaseq)
        if L == 0:
            continue
        if target_len and not (0.4 * target_len <= L <= 1.2 * target_len):
            continue
        if best is None or L > best[2]:
            best = (chain.id, aaseq, L)
    return best

def find_structure_files_for_uid(uid: str) -> List[Path]:
    """
    Look under DOWNLOAD_DIR/<uid>/ and pick:
      - AF2 PDB first: AF-<UID>-*.pdb
      - else any AF2 CIF: AF-<UID>-*.cif
      - else experimental .cif (e.g., 6ABC.cif) as fallback
    Return ordered list of candidate files.
    """
    d = DOWNLOAD_DIR / uid
    if not d.exists():
        return []
    files = list(d.glob("AF-*-model_*.pdb")) + list(d.glob("AF-*.pdb"))
    files += list(d.glob("AF-*-model_*.cif")) + list(d.glob("AF-*.cif"))
    # experimental (RCSB) last
    files += sorted([p for p in d.glob("*.cif") if not p.name.startswith("AF-")])
    # de-dup while preserving order
    seen, out = set(), []
    for p in files:
        if p.exists() and p not in seen:
            out.append(p); seen.add(p)
    return out

def pdb_or_cif_to_3di(uid: str, aa_seq: str) -> Optional[str]:
    """
    Try available files for this UniProt, return a 3Di string (lowercased for consistency),
    preferring AF2 PDB, then AF2 CIF, then experimental CIF.
    """
    candidates = find_structure_files_for_uid(uid)
    if not candidates:
        return None

    for fp in candidates:
        try:
            if fp.suffix.lower() == ".pdb":
                struct = PDB_PARSER.get_structure(uid, str(fp))
            else:
                struct = MMCIF_PARSER.get_structure(uid, str(fp))
        except Exception as e:
            # parsing failed, try next candidate
            continue

        # pick best chain
        target_len = len(aa_seq)
        choice = pick_best_chain_for_uid(struct, target_len)
        if choice is None:
            continue

        chain_id, aaseq_struct, L = choice

        # Encode to 3Di
        try:
            chain = next(c for c in struct.get_chains() if c.id == chain_id)
            s3di = encode_chain_to_3di(chain)
            # optional sanity: if lengths off by a lot, still return but note difference
            return s3di.lower()
        except Exception:
            continue

    return None

# ---------- Build JSONL ----------
OUT_JSONL.parent.mkdir(parents=True, exist_ok=True)
count_ok = 0
count_no_struct = 0

with open(OUT_JSONL, "w", encoding="utf-8") as fout:
    for uid in tqdm(uids, desc="Building JSONL"):
        seq = id2seq.get(uid, "")
        cap = id2cap.get(uid, "")
        if not seq or not cap:
            continue

        s3di = pdb_or_cif_to_3di(uid, seq)
        if s3di is None:
            count_no_struct += 1

        prompt = PROMPT_TMPL.format(seq=seq)
        record = {
            "prompt": prompt,
            "response": cap,          # supervised target
            "aa_seq": seq,            # optional but recommended
            "stru_str": s3di or ""    # optional; empty if unavailable
        }
        fout.write(json.dumps(record, ensure_ascii=False) + "\n")
        count_ok += 1

print(f"\nWrote {count_ok} examples to {OUT_JSONL}")
print(f"Missing structure for {count_no_struct} of them (stru_str left empty).")
